```csharp
# Resolve LightGbmWithOptionsAdvanced Task

Implementing the provided C# code as a dotnet interactive notebook.

## Step 1: Add Task Name and Description

Task Name: Resolve LightGbmWithOptionsAdvanced Task
Task Description: Implement the provided C# code as a dotnet interactive notebook

## Step 2: Install NuGet Package

```nuget
#r "nuget: Microsoft.ML.LightGbm"
```

## Step 3: Import Namespaces

```csharp
using System;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers.LightGbm;
```

## Step 4: Create MLContext

```csharp
var mlContext = new MLContext();
```

## Step 5: Load the Housing Dataset

```csharp
var dataView = Microsoft.ML.SamplesUtils.DatasetUtils.LoadHousingRegressionDataset(mlContext);
```

## Step 6: Split the Data

```csharp
var split = mlContext.Data.TrainTestSplit(dataView, testFraction: 0.1);
```

## Step 7: Create a Pipeline with LightGbm Estimator

```csharp
var labelName = "MedianHomeValue";
var featureNames = dataView.Schema
    .Select(column => column.Name) // Get the column names
    .Where(name => name != labelName) // Drop the Label
    .ToArray();
var pipeline = mlContext.Transforms.Concatenate(
    "Features", featureNames)
    .Append(mlContext.Regression.Trainers.LightGbm(
    new LightGbmRegressionTrainer.Options
    {
        LabelColumnName = labelName,
        NumberOfLeaves = 4,
        MinimumExampleCountPerLeaf = 6,
        LearningRate = 0.001,
        Booster = new GossBooster.Options()
        {
            TopRate = 0.3,
            OtherRate = 0.2
        }
    }));
```

## Step 8: Fit the Pipeline

```csharp
var model = pipeline.Fit(split.TrainSet);
```

## Step 9: Get Feature Importance

```csharp
VBuffer<float> weights = default;
model.LastTransformer.Model.GetFeatureWeights(ref weights);
var weightsValues = weights.DenseValues().ToArray();
Console.WriteLine($"weight 0 - {weightsValues[0]}");
Console.WriteLine($"weight 5 - {weightsValues[5]}");
```

## Step 10: Evaluate the Model

```csharp
var dataWithPredictions = model.Transform(split.TestSet);
var metrics = mlContext.Regression.Evaluate(
    dataWithPredictions,
    labelColumnName: labelName);
```

## Step 11: Print Metrics

```csharp
PrintMetrics(metrics);
```

## Helper Method: PrintMetrics

```csharp
public static void PrintMetrics(RegressionMetrics metrics)
{
    Console.WriteLine("Mean Absolute Error: " + metrics.MeanAbsoluteError);
    Console.WriteLine("Mean Squared Error: " + metrics.MeanSquaredError);
    Console.WriteLine("Root Mean Squared Error: " + metrics.RootMeanSquaredError);
    Console.WriteLine("RSquared: " + metrics.RSquared);
}
```